# Imports

In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

# Web Requests

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

# Get a user's list of games

In [16]:
def getGamesPlayedForUser(userPageURL):
    html_res = BeautifulSoup(
        simple_get(userPageURL),
        'html.parser'
    )
    listOfGameIds = []
    for link in html_res.select('a'):
      if ('href' in link.attrs and '/jsp/game/game' in link['href']):
        listOfGameIds.append(link.text[1:])
    return listOfGameIds
    

In [17]:
listOfGameIds = getGamesPlayedForUser('http://littlegolem.net/jsp/info/player_game_list.jsp?gtid=hex&plid=6398')

In [18]:
print(listOfGameIds)

['1939599', '1938419', '1938296', '1938295', '1938291', '1909790', '1891227', '1877598', '1877596', '1877594', '1877591', '1877587', '1877582', '1877576', '1872780', '1872777', '1861986', '1861985', '1861983', '1861980', '1857774', '1857587', '1855032', '1853841', '1848510', '1845725', '1845724', '1845722', '1845719', '1845715', '1845710', '1818421', '1818420', '1818417', '1818414', '1818410', '1818405', '1818399', '1818392', '1818384', '1789641', '1789639', '1789637', '1789634', '1789630', '1789625', '1789619', '1789612', '1789604', '1789595', '1779168', '1779167', '1779165', '1779162', '1779158', '1779153', '1779147', '1779140', '1771502', '1771501', '1771499', '1771496', '1771492', '1766157', '1766156', '1766153', '1766150', '1766146', '1766141', '1766135', '1766128', '1766120', '1685379', '1676448', '1653132', '1653131', '1651327', '1651326', '1651325', '1651324', '1651323', '1651322', '1651321', '1651320', '1629121', '1629120', '1629119', '1629118', '1621951', '1619428', '1612715'

# Get Information For A Single Game ID

In [22]:
html_res = BeautifulSoup(
        simple_get('http://littlegolem.net/jsp/game/game.jsp?gid='+'1939599'),
        'html.parser'
    )

In [24]:
for div in html_res.select('div'):
    print(div)
    if ('col-xs-6 col-md-6' in div['class']):
        print(div)

<div class="page-header navbar navbar-fixed-top">
<!-- BEGIN HEADER INNER -->
<div class="page-header-inner">
<!-- BEGIN LOGO -->
<div class="page-logo">
<a href="/index.jsp">
<img alt="logo" class="logo-default" src="../../include/logo.png"/>
</a>
<div class="menu-toggler sidebar-toggler hide">
<!-- DOC: Remove the above "hide" to enable the sidebar toggler button on header -->
</div>
</div>
<!-- END LOGO -->
<!-- BEGIN RESPONSIVE MENU TOGGLER -->
<a class="menu-toggler responsive-toggler" data-target=".navbar-collapse" data-toggle="collapse" href="javascript:;">
</a>
<!-- END RESPONSIVE MENU TOGGLER -->
<!-- BEGIN TOP NAVIGATION MENU -->
<div class="top-menu">
<ul class="nav navbar-nav pull-right">
<!-- BEGIN INBOX DROPDOWN -->
<li class="dropdown dropdown-extended dropdown-inbox" id="header_inbox_bar">
<a class="dropdown-toggle" data-close-others="true" data-hover="dropdown" data-toggle="dropdown" href="#">
<i class="fa fa-envelope"></i>
</a>
<ul class="dropdown-menu">
<li>
<p>
				